In [5]:
!kaggle competitions download -c caterpillar-tube-pricing

sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
100%|████████████████████████████████████████| 794k/794k [00:00<00:00, 1.54MB/s]



In [6]:
%%bash
if ls *.zip
then
    for z in *.zip
    do
        unzip "$z"
        rm "$z"
    done
    for c in *.csv
    do
        chmod +rwx "$c"
    done
    if ! test -d csvs
    then
        mkdir csvs
    fi
    mv *.csv csvs/
fi

data.zip
Archive:  data.zip
   creating: competition_data/
  inflating: competition_data/bill_of_materials.csv  
  inflating: competition_data/comp_adaptor.csv  
  inflating: competition_data/comp_boss.csv  
  inflating: competition_data/comp_elbow.csv  
  inflating: competition_data/comp_float.csv  
  inflating: competition_data/comp_hfl.csv  
  inflating: competition_data/comp_nut.csv  
  inflating: competition_data/comp_other.csv  
  inflating: competition_data/comp_sleeve.csv  
  inflating: competition_data/comp_straight.csv  
  inflating: competition_data/comp_tee.csv  
  inflating: competition_data/comp_threaded.csv  
  inflating: competition_data/components.csv  
  inflating: competition_data/specs.csv  
  inflating: competition_data/test_set.csv  
  inflating: competition_data/train_set.csv  
  inflating: competition_data/tube.csv  
  inflating: competition_data/tube_end_form.csv  
  inflating: competition_data/type_component.csv  
  inflating: competition_data/type_connection.

In [8]:
!mv competition_data csvs/

In [11]:
!ls csvs/competition_data/

bill_of_materials.csv comp_other.csv        test_set.csv
comp_adaptor.csv      comp_sleeve.csv       train_set.csv
comp_boss.csv         comp_straight.csv     tube.csv
comp_elbow.csv        comp_tee.csv          tube_end_form.csv
comp_float.csv        comp_threaded.csv     type_component.csv
comp_hfl.csv          components.csv        type_connection.csv
comp_nut.csv          specs.csv             type_end_form.csv


In [144]:
from glob import glob
import pandas as pd
import re


pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

# LOAD ALL DFS

In [193]:
dfs = {}
for path in glob('csvs/competition_data/*.csv'):
    df = pd.read_csv(path)
    pattern = re.compile(r'csvs/competition_data/([^\.]*).*')
    match = pattern.search(path)
    dfs[match.group(1)] = df

train = extract_df(dfs, 'train_set')
test = extract_df(dfs, 'test_set')
len(dfs)

19

In [168]:
for key, val in dfs.items():
    print(key, val.shape)

comp_threaded (194, 32)
comp_adaptor (25, 20)
tube_end_form (27, 2)
comp_straight (361, 12)
comp_tee (4, 14)
comp_boss (147, 15)
components (2048, 3)
comp_float (16, 7)
bill_of_materials (21198, 17)
comp_elbow (178, 16)
type_connection (14, 2)
comp_sleeve (50, 10)
tube (21198, 16)
comp_hfl (6, 9)
type_end_form (8, 2)
comp_other (1001, 3)
type_component (29, 2)
specs (21198, 11)
comp_nut (65, 11)


# CLEAN DFS

In [199]:
a = dfs['specs'].copy()
targets = list(a.columns)
targets.remove('tube_assembly_id')

a['total_specs'] = 0
for target in targets:
    a['total_specs'] += a[target].notnull().astype(int)

In [ ]:
def engineer_specs(specs):
    df = specs.copy()
    targets = list(df.columns)
    targets.remove('tube_assembly_id')
    
    df['total_specs'] = 0
    for target in targets:
        df['total_specs'] += df[target].notnull().astype(int)
    return df

In [179]:
def engineer_tube(tube):
    df = tube.copy()
    target = ['end_a_1x',
              'end_a_2x',
              'end_x_1x',
              'end_x_2x'
             ]
    df['end_totals'] = 0
    for target in targets:
        df[target] = df[target].replace({'N':0, 'Y':1})
        df['end_totals'] += df[target]
    return df

In [107]:
train_franky = franky(dfs, train)
train_franky.shape

(30394, 115)

In [146]:
train_franky.head()

,tube_assembly_id,supplier,quote_date,annual_usage,min_order_quantity,bracket_pricing,quantity,cost,component_id_1,quantity_1,component_id_2,quantity_2,component_id_3,quantity_3,component_id_4,quantity_4,component_id_5,quantity_5,component_id_6,quantity_6,component_id_7,quantity_7,component_id_8,quantity_8,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,end_x_2x,end_a,end_x,num_boss,num_bracket,other,spec1,spec2,spec3,spec4,spec5,spec6,spec7,spec8,spec9,spec10,component_id,component_type_id,connection_type_id,intended_nut_thread,intended_nut_pitch,unique_feature,plating,orientation,weight,hex_nut_size,seat_angle,thread_size,thread_pitch,blind_hole,adaptor_angle,overall_length,hex_size,end_form_id_1,connection_type_id_1,length_1,thread_size_1,thread_pitch_1,nominal_size_1,end_form_id_2,connection_type_id_2,length_2,thread_size_2,thread_pitch_2,nominal_size_2,end_form_id_3,connection_type_id_3,length_3,thread_size_3,thread_pitch_3,nominal_size_3,end_form_id_4,connection_type_id_4,length_4,thread_size_4,thread_pitch_4,nominal_size_4,bolt_pattern_long,bolt_pattern_wide,head_diameter,thickness,mj_class_code,groove,extension_length,drop_length,mj_plug_class_code,type,outside_shape,base_type,height_over_tube,base_diameter,shoulder_diameter,name,elbow_angle,plug_diameter,hose_diameter,corresponding_shell,coupling_class,material,part_name,end_form_id,forming
0,TA-00002,S-0066,2013-07-07,0,0,Yes,1,21.905933,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SP-0019,6.35,0.71,137.0,8,19.05,N,N,N,N,EF-008,EF-008,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TA-00002,S-0066,2013-07-07,0,0,Yes,2,12.341214,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SP-0019,6.35,0.71,137.0,8,19.05,N,N,N,N,EF-008,EF-008,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,TA-00002,S-0066,2013-07-07,0,0,Yes,5,6.601826,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SP-0019,6.35,0.71,137.0,8,19.05,N,N,N,N,EF-008,EF-008,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,TA-00002,S-0066,2013-07-07,0,0,Yes,10,4.687770,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SP-0019,6.35,0.71,137.0,8,19.05,N,N,N,N,EF-008,EF-008,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TA-00002,S-0066,2013-07-07,0,0,Yes,25,3.541561,C-1312,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SP-0019,6.35,0.71,137.0,8,19.05,N,N,N,N,EF-008,EF-008,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [110]:
def pp_1(df):
    df = df.copy()
    df['year'] = newFeature_year(df, 'quote_date')
    df['month'] = newFeature_month(df, 'quote_date')
    df.drop('quote_date', axis=1)
    
    df = newFeature_isMISSING(df, df.columns)
    return df

In [103]:
########### FEATURE EXTRACTION #################

def newFeature_year(df, col):
    df = df.copy()
    df[col] = pd.to_datetime(df[col], infer_datetime_format=True)
    return df[col].dt.year
def newFeature_month(df, col):
    df = df.copy()
    df[col] = pd.to_datetime(df[col], infer_datetime_format=True)
    return df[col].dt.month
def newFeature_isMISSING(df, cols):
    df = df.copy()
    for col in cols:
        df[col + '_IS_MISSING'] = df[col].isnull().astype(int)
    return df

In [133]:
def features_byNaN(df, maxNaN_allowed):
    se = df.isna().sum().sort_values()
    features = se.where(se < maxNaN_allowed).dropna().index
    return features

In [95]:
def franky(dfs, root):
    dfs = dfs.copy()
    stitch = root
    while (len(matching_columns(stitch, dfs)) > 0):
        for match in matching_columns(stitch, dfs):
            df = extract_df(dfs, match['key'])
            stitch = merge_dfs(stitch, df)
    return stitch

In [136]:
########## FRANKY HELPERS ###############

def matching_columns(df, list_of_dfs):
    matches = []
    for key,val in list_of_dfs.items():
        if set(df.columns) & set(val.columns):
            matches.append({'key': key, 'len': len(val.columns), 'shape' : val.shape})
    return matches

def merge_dfs(df1, df2):
    df1 = df1.merge(df2, how='left')
    return df1

def extract_df(dfs, key):
    try: 
        df = dfs[key]
        del dfs[key]
        return df
    except Exception as e:
        print(e)